In [29]:
from enum import Enum
from typing import Dict, List
from urllib import request

import re

import pandas as pd

In [30]:
OBSERVERS = [
    "8500LOEL",
    "8930MK",
    "8660BW",
    "8380NKBJ",
]

In [31]:
class Season(Enum):
    SPRING = 1
    FALL = 2

In [32]:
def get_observations(
    observer: str,
    year: int,
    season: Season,
) -> pd.DataFrame:
    """Query DOFbasen for observations by a given observer in a given year and
    season. Returns a pandas DataFrame with the result."""
    
    url = "&".join(
        [
            "https://dofbasen.dk/excel/search_result1.php?design=excel",
            "soeg=soeg",
            "periode=maanedaar",
            f"aar_first={year}",
            f"aar_second={year}"
            f"halvaar={season.value}",
            "obstype=observationer",
            "species=alle",
            f"obserdata={observer}",
            "textsearch=Boligbirding",
            "sortering=dato"
        ]
    )

    data = [
        line.decode("ISO-8859-1").strip().split(";")
        for line in request.urlopen(url)
    ]

    # Output from DOFbasen is in an outdated format with a strange encoding,
    # so we must do some manual clenup here and there. This parts fixes issues,
    # where the user has entered a newline in the comment field.

    sanitized_data: List[str] = []

    # Add header to sanitized data.
    sanitized_data.append(data[0])

    def is_date(string: str) -> bool:
        """Helper function to check if a string is a date in the format YYYY-MM-DD"""
        return re.match(r"^\d{4}-\d{2}-\d{2}$", string)

    i = 1  # First data row in data.

    while i < len(data):

        # We have reached the end of the data and there is nothing to peek at.
        if i == len(data) - 1:
            sanitized_data.append(data[i])
            break

        # If row does not start with a date, it is not a valid entry.
        if not is_date(data[i][0]):
            i += 1
            continue

        # Row starts with a data. Have a peak at the following row. If it does not start
        # with a date, is is a continuation of the current row.
        if not is_date(data[i+1][0]):
            merged_entry = data[i][:-1] + [data[i][-1] + data[i+1][0]] + data[i+1][1:]
            sanitized_data.append(merged_entry)
            i += 1
            continue
        
        # Row starts with a date and the following row also starts with a date,
        # so everything is fine.
        sanitized_data.append(data[i])
        i += 1

    return pd.DataFrame(dict(zip(sanitized_data[0], sanitized_data[i])) for i in range(1, len(sanitized_data)))


In [33]:
def get_first_observations(
    observer: str,
    year: int,
    season: Season,
) -> Dict[str, str]:
    """Get the first observation of each species by a given observer in a given
    year and season. Returns a dictionary with species names as keys and dates
    as values."""
    
    data = get_observations(
        observer=observer,
        year=year,
        season=season,
    )
    return {
        species.replace('"', ''): data[data["Artnavn"] == species]["Dato"].min() for species in data["Artnavn"].unique()
    }

In [34]:
def create_table(
    observers: List[str],
    year: int,
    season: Season,
) -> pd.DataFrame:
    """Create a table with the first observations of each species by a list of
    observers in a given year and season. Returns a pandas DataFrame with the
    result."""    

    return pd.DataFrame({observer: get_first_observations(observer, year, season) for observer in observers})

In [35]:
# Example usage:

create_table(
    observers=OBSERVERS,
    year=2023,
    season=Season.FALL,
).to_csv("example.tsv", sep="\t")